# 

In [1]:
%config Completer.use_jedi = False

import configparser
config = configparser.ConfigParser()
config.read('har-config.txt')

source_folder = config.get('Configuration', 'source_folder')
time_step = int(config.get('Configuration', 'time_step'))
window_size = int(config.get('Configuration', 'window_size'))
data_folder = config.get('Configuration', 'data_folder')
ini_date = config.get('Configuration', 'ini_date')
end_date = config.get('Configuration', 'end_date')


anchors=config.get('Configuration', 'anchors')
anchors = [(item.strip()) for item in anchors.split(',')]

w = int(config.get('Configuration', 'w'))
h = int(config.get('Configuration', 'h'))



In [2]:
import pandas as pd
import collections
import math
import os

import cv2
import matplotlib.pyplot as plt
import cv2
import sys
import numpy as np
import math
import time
from PIL import Image 
import shutil

from pandas import read_csv  



In [3]:
import numpy as np
from pandas import read_csv    
import pandas as pd
import time

from datetime import datetime

off_zone=60*60*2

def day_time(ti):
    return int((int)((ti+off_zone)/(60*60*24)))


def time2str(tt):
    return datetime.fromtimestamp(tt).strftime("%Y-%m-%d %H:%M:%S")

tN = (int)(datetime.strptime(end_date, '%Y-%m-%d %H:%M:%S').timestamp())
print (end_date,"End date is", tN, "day:",day_time(tN), time2str(tN))


print(ini_date)
t0 = (int)(datetime.strptime(ini_date, '%Y-%m-%d %H:%M:%S').timestamp())
print (ini_date,"Init date is", t0, "day:",day_time(t0), time2str(t0))


ts=list(range(t0,tN,time_step))

def day_time0(ti):
    return day_time(ti)-day_time(t0)+1

def relT(ti):
    return (int)((ti-t0)/time_step)

def absT(tt):
    return (int)(tt*time_step+t0)

day=list(range(day_time(t0),day_time(tN)+1))[0]

from datetime import datetime, timedelta


def getStrDatefrom(day_number):
    # Reference date (1970-01-01)
    reference_date = datetime(1970, 1, 1)
    
    # Compute the date corresponding to the day number
    resulting_date = reference_date + timedelta(days=day_number)
    
    return resulting_date.strftime("%Y-%m-%d")




day_str=getStrDatefrom(day)
print(day_str)
users=config.get('Configuration', day_str)
users = [(item.strip()) for item in users.split(',')]


print(users)


2024-08-01 01:59:59 End date is 1722470399 day: 19936 2024-08-01 01:59:59
2024-07-31 02:00:00
2024-07-31 02:00:00 Init date is 1722384000 day: 19935 2024-07-31 02:00:00
2024-07-31
['16fe', '5b66', 'ed9c']


In [4]:
def getFPImg(tt):
    #print(source_folder+"/DAY_"+str(day)+"/FP."+str(tt)+".png")
    return np.array(Image.open(source_folder+"/DAY_"+str(day)+"/FP."+str(tt)+".png"))/255.0



In [5]:

rssi0={}


T0={}
T1={}





anchors_data={}
for user in users:

    rssi0[user]=[]
    T1[user]=[]
    T0[user]=[]    


    anchors_data[user]={}
    for anchor in anchors:
        #print(source_folder+"/DAY_"+str(day)+"/"+user+"."+anchor+".tsv")
        #print(user,anchor)
        series=read_csv(source_folder+"/DAY_"+str(day)+"/"+user+"."+anchor+".tsv", sep="\t",parse_dates=True, header=None, names=["time","d","d2"])
        series=series.values
        anchors_data[user][anchor]=series


for it,tt in enumerate(ts):
    #print(tt)
    #imgX=getFPImg(tt)
    #print("imgX",imgX.shape)

    #img00=np.zeros((w,h))
    #scene=getScene(tt)
    #if(scene):
    #    img00=getSceneImg(scene)
    #print("img00",img00)
    #imgX=imgX-img00
    #imgX[imgX<0]=0
    for user in users:
        dist=[]
        for ixa,anchor in enumerate(anchors):
            #print(tt,time2str(tt),anchor,user)
            d=anchors_data[user][anchor][it][1]
            if(d>=0):
                if(d<10):
                    d=(1-d/10.0)
                else:
                    d=0
            else:
                    d=0

            dist.append(d)
        for ixa,anchor in enumerate(anchors):
            #print(tt,time2str(tt),anchor,user)
            d=anchors_data[user][anchor][it][2]
            #rint(anchors_data[user][anchor][it][0], "vs",tt)

            #if(d!=-1):
            #    d=(d - rssi_min) / (rssi_max - rssi_min)

            #print(anchors_data[user][anchor][it][0],anchors_data[user][anchor][it][1],anchors_data[user][anchor][it][2])
            if(d>=0):
                if(d<10):
                    d=(1-d/10.0)
                else:
                    d=0
            else:
                    d=0

            #print("\t anchor:",anchor, anchors_data[user][anchor][it], "tt:",tt)
            dist.append(d)

        dist=np.array(dist)
        #print("dist:",dist)

        rssi0[user].append(dist.reshape(len(anchors*2),1))
        #XY0[user].append(imgX[:, :, np.newaxis])

        T0[user].append(it)
        T1[user].append(tt)

        


In [6]:
from collections import Counter

for user in users:
    #XY0[user]=np.array(XY0[user])
    rssi0[user]=np.array(rssi0[user])

    T0[user]=np.array(T0[user])
    T1[user]=np.array(T1[user])
    minT=np.min(T0[user])
    maxT=np.max(T0[user])


    print(minT)
    print(maxT)

0
86398
0
86398
0
86398


In [7]:
#XYT=[]
rssiT=[]
YT=[]
TT=[]

window_size=6

T=window_size

TA=math.ceil(window_size/2)
TB=math.floor(window_size/2)


print(TA,TB)


T=window_size

K=len(anchors*2)

SS=[]
U=[]

imgZero=np.zeros((T,h, w,1))
seqZero=np.zeros((T,K,1))
for user in users:
    minT=np.min(T0[user])
    maxT=np.max(T0[user])
    for t0 in range(minT+TA,maxT-TB):        
        U.append(user)
        
        #print(user,t0)

        
        tt=T1[user][t0]
        TT.append(tt)

        if(all(map(lambda x: x <= 0, rssi0[user][t0-TA:t0+TB].flatten()))):
            #print("Distance -1 for RSSI in sliding window!",t0)
            #XYT.append(imgZero)
            rssiT.append(seqZero)            
            SS.append(False)
            continue

 

       # if(XY0[user][t0-TA:t0+TB].shape[0]!=T):
       #     #print("#",XY0[user][t0-TA:t0+TB].shape)
        #   #XYT.append(imgZero)
       #     rssiT.append(seqZero)            
       #     SS.append(False)
       #     continue
        if(rssi0[user][t0-TA:t0+TB].shape[0]!=T):
            #print("@",rssi0[user][t0-TA:t0+TB].shape)
            #XYT.append(imgZero)
            rssiT.append(seqZero)            
            SS.append(False)
            continue

        #XYT.append(XY0[user][t0-TA:t0+TB])
        rssiT.append(rssi0[user][t0-TA:t0+TB])
        SS.append(True)
#XYT=np.array(XYT)
rssiT=np.array(rssiT)
SS=np.array(SS)    
U=np.array(U)    

#print(XYT.shape)
print(rssiT.shape)
print(SS.shape)


3 3


(259176, 6, 18, 1)
(259176,)


In [8]:
#print(XYT.shape)
print(rssiT.shape)
print(SS.shape)

(259176, 6, 18, 1)
(259176,)


In [9]:
from tensorflow import keras

from tensorflow.keras import layers
from keras.optimizers import Adam
from keras.layers import MaxPooling2D,Activation, Dense

from tensorflow import Tensor
from tensorflow.keras.layers import Softmax,Input, Conv1D, Conv2D, ReLU, BatchNormalization,\
                                    Add, AveragePooling2D, Flatten, Dense,Dropout,LSTM,TimeDistributed,MaxPooling1D
from tensorflow.keras.models import Model

from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report 
from keras.losses import mean_squared_error

from keras import backend as K

from sklearn.model_selection import KFold

from tensorflow import keras

from tensorflow.keras import layers
from keras.optimizers import Adam
from keras.layers import MaxPooling2D,Activation, Dense

from tensorflow import Tensor
from tensorflow.keras.layers import Lambda, Dot,Reshape,Multiply, Input, Conv1D, Conv2D, ConvLSTM2D,ReLU,UpSampling2D, BatchNormalization,\
                                    Add, AveragePooling2D, Flatten, Dense,Dropout,LSTM,TimeDistributed,MaxPooling1D, Concatenate
from tensorflow.keras.models import Model, Sequential

from keras import backend as K
import math

from tensorflow.keras.losses import CategoricalCrossentropy

2024-08-29 14:28:09.593626: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-29 14:28:09.811058: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


2024-08-29 14:28:11.282611: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [10]:
from keras.models import load_model

autoencoder = load_model("autoencoder.model3.all.50.h5")
autoencoder.summary()

2024-08-29 14:28:14.403230: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-29 14:28:14.536614: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-29 14:28:14.536660: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-29 14:28:14.538909: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:982] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-08-29 14:28:14.538951: I tensorflow/compile

2024-08-29 14:28:16.033259: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-08-29 14:28:16.034126: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-08-29 14:28:16.035125: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Model: "model"


_________________________________________________________________


 Layer (type)                Output Shape              Param #   


...

 cropping2d (Cropping2D)     (None, 18, 22, 1)         0         


Total params: 13,810,561


Trainable params: 13,810,561


Non-trainable params: 0


_________________________________________________________________


In [11]:
from tensorflow.python.framework import ops
from keras.models import load_model


In [12]:

batch=1024

for user in users:
    os.makedirs(data_folder+"/DAY_"+str(day)+"/"+user, exist_ok=True)

imgZero=np.zeros((h, w,1))

for ix in range(0,rssiT.shape[0],batch):
    L=batch
    if(ix+batch>rssiT.shape[0]):
        L=rssiT.shape[0]-ix
    print(ix,L)
    test_indices=np.array(list(range(ix,ix+L,1)))
    #print(test_indices)

    #XYT_test=XYT[test_indices]
    rssiT_test=rssiT[test_indices]
    
    YT_prediction=autoencoder.predict(rssiT_test)
                           
    print("prediction shape:",YT_prediction.shape)
    import matplotlib.pyplot as plt

    #K=5
    #ks=np.random.randint(0, len(test_indices), K)

    # Create a figure with two subplots (one row, two columns)
    #fig, axes = plt.subplots(K, 1, figsize=(20, 20))

    M=w
    N=h


    #for k in range(0,K):
    #    mpOne=YT_prediction[ks[k]].reshape(N,M)

     #   axes[k].imshow(mpOne, cmap='viridis', interpolation='nearest')
     #   axes[k].set_title('True (last frame) of player')
     #   axes[k].axis('off')


    for jx in range(0,L):
            img0=YT_prediction[jx]
            kx=ix+jx
            user=U[kx]
            tt=TT[kx]
            ok=SS[kx]
            #print("\t # ", kx, "tt:",tt,time2str(tt), "user:",user,ok)
            if(ok):
                cv2.imwrite(data_folder+"/DAY_"+str(day)+"/"+user+"/FP."+str(tt)+".png", img0*255) 
            else:
                cv2.imwrite(data_folder+"/DAY_"+str(day)+"/"+user+"/FP."+str(tt)+".png", imgZero) 



0 1024


2024-08-29 14:28:18.348702: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-08-29 14:28:18.349474: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-08-29 14:28:18.350527: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

2024-08-29 14:28:20.164488: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:424] Loaded cuDNN version 8600


 1/32 [..............................] - ETA: 2:16

2024-08-29 14:28:22.430422: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:637] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


...

2024-08-29 14:41:28.762419: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-08-29 14:41:28.763495: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-08-29 14:41:28.764240: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

1/4 [======>.......................] - ETA: 0s

4/4 [==============================] - ETA: 0s

4/4 [==============================] - 1s 151ms/step


prediction shape: (104, 18, 22, 1)
